# Importing/Storing data

In [1]:
# pip install wfdb
# pip install 

In [2]:
import wfdb
import os
import glob
import pandas as pd
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt

In [3]:
#print('Current directory is: {}'.format(os.getcwd()))

In [4]:
records = []
properties = []
annot = []
AnnSymb = []
AnnSamp = []
AnnRhythm = []
Rpeak_Samp = []
Rpeak_Symb = []


for f in glob.glob('C:/Users/masud/Desktop/Github_Bachelor_Project/wfdb-python/data/*.dat'): ##### change the path (afib) to your own directory. The rest should 
                                                                                             ##### remin the same
    sig, fields = wfdb.rdsamp(f[:-4])  #### In this function, you have the opportunity to pass 
                                       #### "channels=[0]" or "channels=[1]" to select channel 1 or 2.
    ann = wfdb.rdann(f[:-4], 'atr')
    QRS = wfdb.rdann(f[:-4], 'qrs')
    Symb = pd.Series(ann.symbol)
    Samp = pd.Series(ann.sample)
    QRS_Symb = pd.Series(QRS.symbol)
    QRS_Samp = pd.Series(QRS.sample)
    Rhythm = pd.Series(ann.aux_note)
    records.append(sig)
    properties.append(fields)
    annot.append(ann)
    AnnSymb.append(Symb)
    AnnSamp.append(Samp)
    Rpeak_Symb.append(QRS_Symb)
    Rpeak_Samp.append(QRS_Samp)
    AnnRhythm.append(Rhythm)
    
    

AnnSymb = pd.Series(AnnSymb).values
AnnSamp = pd.Series(AnnSamp).values

In [71]:
#This is only for last record, calculated in above loop
print(AnnRhythm[22])
print("------------------------------------")
print(AnnSamp[22])
print("------------------------------------")
print("All samples of R-peaks below")
print(Rpeak_Samp[22])
print("------------------------------------")
print(records[22])
print(type(records[22]))
#print(np.where(Rpeak_Samp[22] <= ))
#df.iloc[22] # acessing ROW 22

0       (N
1    (AFIB
2       (N
3    (AFIB
dtype: object
------------------------------------
0         96
1    2808396
2    2820743
3    2850351
dtype: int32
------------------------------------
All samples of R-peaks below
0            193
1            390
2            586
3            782
4            905
          ...   
59547    8999654
59548    8999811
59549    8999879
59550    8999948
59551    9000001
Length: 59552, dtype: int32
------------------------------------
[[-0.275 -0.37 ]
 [-0.245 -0.45 ]
 [-0.305 -0.43 ]
 ...
 [-0.24  -0.52 ]
 [-0.3   -0.52 ]
 [-0.27  -0.565]]
<class 'numpy.ndarray'>


# Working oon...

In [32]:
#x = np.random.rand(100)
rec = Rpeak_Samp[22]
samp = AnnSamp[22]
#label_column = np.where((rec > samp[0]) & (rec < samp[1]), '(N', 'NaN')
#label_column = np.where((rec > samp[1]) & (rec < samp[2]), '(AFIB', 'NaN')
#df = pd.DataFrame(Rpeak_Samp[22])
#df['Label'] = label_column
#df

In [40]:
#maksing
#rec[(rec > samp[0]) & (rec < samp[1])]

df1 = pd.DataFrame(rec[(rec > samp[0]) & (rec < samp[1])])
df1['Label'] = '(N'
df1

df2 = pd.DataFrame(rec[(rec > samp[1]) & (rec < samp[2])])
df2['Label'] = '(AFIB'
df2

df3 = pd.concat([df1, df2])
df3

,0,Label
0,193,(N
1,390,(N
2,586,(N
3,782,(N
4,905,(N
...,...,...
15216,2820185,(AFIB
15217,2820295,(AFIB
15218,2820388,(AFIB
15219,2820505,(AFIB


In [69]:
from window_slider import Slider
list = records[22]
bucket_size = 6600
overlap_count = 3300
slider = Slider(bucket_size,overlap_count)
slider.fit(list)       
while True:
    window_data = slider.slide()
    # do your stuff
    
    # do you stuff
    print(window_data)
    if slider.reached_end_of_list(): break

ValueError: Bucket size should be smaller than list size

### Visualising the ECG signals

In [49]:
#plt.title("Both Channels (0 and 1)")
#plt.title("Channel 0 of the ECG") #Called ['ECG1']
#plt.title('Channel 1 of the ECG') #Called ['ECG2']
#plt.plot(sig[:500])

### Accessing annotations 

In [9]:
#display(fields) ### Last record annotation
#print(f"Units in which the ECG is measured {fields['units']}")

In [10]:
#display(properties[22])
#print(f"Units in which the ECG is measured {properties[22]['units']}")

# Calculating the R to R Intervals

In [11]:
single_record_rri = []
rri = []

squared_summed_differencies = 0
i = 0
for i in range(len(Rpeak_Samp)): ### loops 23 times
    single_record_rri = np.diff(Rpeak_Samp[i])
    rri.append(single_record_rri)

In [12]:
#A sample of 250 elements, gives 1 clear signal (because 250hz frequency)
#x = np.arange(0,250)
#plt.plot(x,rri[22][750:1000])

# Segmenting the records and calculating features

In [13]:
#seg_all_rec = []

#amount_to_segment = -(len(rri[0]) % 20)
#record_to_segment = rri[0][:amount_to_segment]
#length = len(record_to_segment) / 20
#segmented_record = np.split(record_to_segment,length)
#seg_all_rec.append(segmented_record)

In [14]:
single_record_mean = []
rri_mean = []
single_record_std = []
rri_std = []
single_record_RMSSD = []
RMSSD = []
single_record_nRMSSD = []
nRMSSD = []

all_segmented_records = []
all_features = []


### TO acess every single record
### record_to_segment = rri[index]
for index in range(len(rri)): ### Loops 23 times
    #(below line) gives a value between 0 and 19, which is the amount used to remove from the end of a record, by slicing.
    amount_to_segment = -(len(rri[index]) % 20)
    if amount_to_segment != 0:
        rri_segmented_record = rri[index][:amount_to_segment]
    else: 
        rri_segmented_record = rri[index] #if divisible by 20, no need to slice this record
    #(below line) gives us the number of "blocks" to put every single 20-segment into.
    length = len(rri_segmented_record) / 20
    rri_segmented_record = np.split(rri_segmented_record,length)
    all_segmented_records.append(rri_segmented_record)
    
    features = []
    
    ### Calculating features for every single segmented "block" inside the segmented_record variable
    for i in range(len(rri_segmented_record)):
        #MEAN
        f1 = np.mean(rri_segmented_record[i])
        #STD
        f2 = np.std(rri_segmented_record[i])
        #RMSSD
        sqaured_summed_differencies = 0
        for k in range(len(rri_segmented_record[i]) - 1): ### loops 19 times
            sqaured_summed_differencies += (rri_segmented_record[i][k] - rri_segmented_record[i][k+1])**2
        squared_summed_differencies_multiplied = 1/(len(rri_segmented_record) - 1) * sqaured_summed_differencies
        f3 = np.sqrt(squared_summed_differencies_multiplied)
        #NORMALIZED RMSSD
        f4 = (f3 / f1)
        
        my_features = pd.Series([np.around(f1,3), np.around(f2,3), np.around(f3,3), np.around(f4,3)], index=['Mean','STD','RMSSD','Normalized RMSSD'])
        features.append(my_features)
        
    all_features.append(features)

### Features for a single block (20 heart beats), inside a given record
### Accessing the 23 individual records (index 0 to 22), gives a single record, that is segmented into arrays of 20. Accessing each 20 array segment/block, by indexing, this time second dimension [ ]-->[ ]<--

In [15]:
n = 3
m = 156

print(f"Record nr {n}, and index {m} in that record contains this segment:") 
print(all_segmented_records[n][m])
print("Features for that specific segment:") 
print(all_features[n][m])

Record nr 3, and index 156 in that record contains this segment:
[165 175 173 170 162 163 165 169 167 169 164 165 162 168 170 169 169 165
 168 169]
Features for that specific segment:
Mean                167.350
STD                   3.380
RMSSD                 0.379
Normalized RMSSD      0.002
dtype: float64


### conversion to numpy arrays

In [18]:
#print(len(all_segmented_records[17]))
#print(len(all_features[18]))

In [19]:
#segmented_arr = np.asarray(all_segmented_records[17])
#feature_arr = np.asarray(all_features[17])
#feature_arr

# Classifications algorithms

In [20]:
#from sklearn.model_selection import train_test_split
#X = all_features[3]

In [21]:
#from sklearn import svm
#X = [[0, 0], [1, 1]]
#y = [0, 1]
#X = all_features[3]
#y = np.arange(2167)
#clf = svm.SVC()
#clf.fit(X, y)

In [22]:
#Z = np.asarray(all_features[18])
#predicted = clf.predict(Z)